In [1]:
###IMPORTS###
#to run a server, config allows us to change the general look, allows use of CSS
from pywebio import start_server, config
#allows input fields to remain on page after submission (Makes fields persistent)
from pywebio.pin import *
#allows use of input commands
from pywebio.input import *
#allows use of output commands
from pywebio.output import *

import Database as db


In [2]:
"""  
Programmers: Kellie Glasgow & Courtney Ward  
Languages: python, CSS  
Tools: pywebio, pandas, excel  
Runs on: Visual Studio Code with Jupyter  
Included Files: cookbook.xlsx  
Purpose: Create a cookbook application to add & view recipes in your own personalized recipe book.  
Pseudocode  
Main menu has two buttons to navigate to two features  
    Add recipe  
    View Recipes  
View Recipes  
    Access database to get recipe names & basic information  
    Display this information in a table  
    Dropdown menu to select recipe  
    Submit button to view selected recipe  
View Selected Recipe  
    Access database & display recipe information, ingredients, & steps  
Add Recipe  
    Have input fields to collect data about recipe from user  
    Validate & save data  
""" 

###IMPORTS### 
#to run a server, config allows us to change the general look, allows use of CSS 
from pywebio import start_server, config 
#allows use of input commands 
from pywebio.input import * 
#allows use of output commands 
from pywebio.output import * 
#Import DBModel to access DB 
import Database as AccessDB 
 
#Declaring & initializing global styles as string of CSS code to keep consistent formatting throughout 

headerStyle = 'text-align: center; background-color: tan; padding-top: 40px; padding-bottom: 40px; color: brown; font-size: 50px; font-weight: bold' 
navButtonStyle = 'text-align: left; margin-top: 20px' 
headingStyle = 'text-align: center; color:black; font-size: xx-large; font-weight: bold' 
promptStyle = 'text-align: center; color:black; font-size: x-large; font-weight: bold' 
recipeHeadingStyle = 'margin-left: 400px; color: black; font-size: 40pt; font-weight: bold' 
recipeDescriptionStyle = '' 
recipeStatsStyle = '' 
recipeIngredStyle = 'margin-left: 100px; font-size 30pt' 
recipeDirectionStyle = 'margin-left: 100px; font-size = 35pt; font-weight: bold' 
centerStyle = 'text-align: center' 

###Start Function### 
def start_app(): 
    AccessDB.open_cookbook() 
    main_menu() 
###VIEW FUNCTIONS### 
def print_header(): 
    ###Function to print running header & nav bar###    
    # Nav bar button to return to main menu  
    put_button(["Main Menu"], onclick=main_menu).style('text-align: left; margin-top: 20px') 
    # App title header 
    put_markdown('# The Busy Chef').style(headerStyle) 
def main_menu(): 
    ###Layout for main menu### 
    #Clear any prior output 
    clear() 
    with use_scope("main_menu", clear = True): 
        #display main menu header at top of page 
        put_markdown("# Welcome to the Busy Chef").style(headerStyle) 
        #Prompt user to make selection 
        put_text("What would you like to do?").style(promptStyle) 
        #Define & layout buttons for options 
        put_buttons(["View Recipes", 'Add Recipe'], onclick=menuButtons).style('text-align: center') 

def menuButtons(btn_val): 
    ###Tells main menu buttons what to do/what functions to call### 
    #Launches View Recipes Page 
    if btn_val == "View Recipes": 
        view_recipes() 
    #Launches Add recipe page 
    elif btn_val == "Add Recipe": 
        add_recipe() 

def view_recipes(): 
    ###Layout & data for View All Recipes screen### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    put_text("Recipes").style(recipeHeadingStyle) 
    #Get list of recipe names from DB 
    optionsList = AccessDB.getRecipeList() 
    #Populate DDL 
    response = select('Which recipe would you like to view?', optionsList) 
    display_recipe(response) 

def display_recipe(response): 

    ###Get data for recipe and format & display it to page### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    #Get data for recipe 
    recipe = AccessDB.getRecipeInformation(response) 
    #format & display recipe 
    #Recipe Name 
    put_text(recipe['RecipeName']).style(headingStyle) 
    #Recipe Description 
    put_text(recipe['Description']).style('recipeDescriptionStyle') 
    #Recipe General Info/Stats
    #Ingredients 
    put_text('Ingredients').style(headingStyle) 
    put_table(recipe['ingredients']).style('text-align: center')
    stepNum = 1
    for step in recipe['instructions']:
        txt = "{}. {}".format(stepNum,step)
        put_text(txt)
    #Directions 

def get_ingredients(num): 
    ###Layout for user input screens to gather new recipe ingredients### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    #String for recipe ingredient prompt with automatic incremented ingredient number 
    inputStr = "Enter Ingredient Information for Ingredient {}".format(num + 1) 
    #Displays input field & prompt for recipe ingredient, amount & measurement 
    ingred_info = input_group(inputStr, [ 

        input('Ingredient Name', name = 'Name', type = TEXT), 

        input('Ingredient Amount', name = 'Amount', type = TEXT), 

        input('Ingredient Measurement(example: cup, teaspoon)', name ='Measurement', type = TEXT) 
    ]) 
    #Update ingredient data for recipe  
    ingredHelp = {'IngredListID':len(recipedf)} 
    ingred_info.update(ingredHelp) 
    return ingred_info 

def get_steps(num): 

    ###Layout for user input screens to gather new recipe steps### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header() 
    #String for recipe step prompt with automatic incremented step number 
    inputStr = "Enter Step {}".format(num + 1) 
    #Displays input field & prompt for recipe step  
    info = input_group(inputStr, [ 

        input('Instruction', name = 'Instruction', type = TEXT) 

    ]) 
    step = num + 1 
    step = str(step) + '.' 
    #Update step data for recipe  
    stepsHelp = {'StepListID':len(recipedf),'StepNum': step} 
    info.update(stepsHelp) 
    return info 

def add_recipe(): 
    ###Layout for user input screen to gather new recipe information### 
    #Clear any prior output & display common header at top of page 
    clear() 
    print_header()
    #get information from user 
    info = input_group("Please Enter your Recipe Information",[ 
        input('Enter Recipe Name', name = 'RecipeName', type = TEXT), 
        input('Enter Recipe Description', name = 'Description', type = TEXT), 
        radio('Select your Tags', options = ['Vegetarian', 'Heart Healthy', 'Protien Heavy', 'Party'], name = 'tags'),       
        input('Food Category (Example: Indian, American, Italian)', name = 'foodCat', type = TEXT), 
        select('Cuisine', options = ['Breakfast', 'Brunch', 'Lunch', 'Dinner', 'Appetizer', 'Dessert'], name  = 'cuisine'), 
        input('Enter the Prep Time for the Meal', name = 'prepTime', type = NUMBER), 
        input('Enter the Cook Time for the Meal', name = 'cookTime', type = NUMBER), 
        input('Enter the number of ingredients in recipe:', name = 'numIngred', type = NUMBER), 
        input('Enter the number of steps in recipe:', name = 'numInstruct', type = NUMBER) 
    ]) 

    #Getting dictionary of db attribute names for consistency & avoid db errors 
    recipeHelp = {'RecipeID': len(recipedf) + 1} 
    #Update dataframe with recipe input 
    info.update(recipeHelp) 
    recipedf.loc[len(recipedf) + 1] = info 
    #Get each ingredient info 
    for i in range(info['numIngred']): 
        ingred = get_ingredients(i) 
        ingredientsdf.loc[len(ingredientsdf) + 1] = ingred 
    #Get each step info 
    for i in range(info['numInstruct']): 
        steps = get_steps(i) 
        stepsdf.loc[len(stepsdf) + 1] = steps 
    #Updating dataframe 
    recipedf.index = recipedf.index + 1 
    ingredientsdf.index = ingredientsdf.index + 1 
    stepsdf.index = stepsdf.index + 1 
    global dfDisplay  
    dfDisplay = recipedf.drop(columns=['RecipeID', 'numIngred','numInstruct']) 
    dfDisplay.rename(columns={'RecipeName': 'Recipe', 'Description': 'Description', 'tags':'Tag', 'foodCat':'Category', 'cuisine': 'Cuisine', 'prepTime': 'Prep Time', 'cookTime': 'Cook Time'}, inplace=True) 
    ###Save data### 
    save_dfs() 
    ###Go to main menu after submission### 
    main_menu() 
    ###PYWEBIO FUNCTION THAT OPENS UNUSED PORT ON NETWORK### 

###Server starts with main_menu page### 

start_server(start_app, port = 8080, debug = True ) 

Running on all addresses.
Use http://10.57.193.193:8080/ to access the application


RuntimeError: This event loop is already running